In [ ]:
from Node import Node

#given code for an outline of the greedy algothrim 
def filter_uppercase_and_spaces(input_string: str) -> str:
    """
    Filters the input string to retain only uppercase letters and spaces.
    """
    return "".join(
        char for char in input_string.upper() if char.isalpha() or char == " "
    )


def count_frequencies(input_string: str) -> list[int]:
    """
    Counts the frequency of each uppercase letter in the input string.
    Returns a list of 26 integers, where index 0-25 correspond to 'A'-'Z'.
    You can assume the input string contains only uppercase letters and spaces.
    And that spaces are the most frequent character, so really we dont need
    to count them.
    """
    frequencies = [0] * 27 # frequency list for the 26 letters and space
    for char in input_string: #count all char apperances 
        if char == " ":
            frequencies[26] += 1
        elif char.isalpha():
            index = ord(char) - ord("A") #convert letter to 0-25 index
            frequencies[index]+=1
    


def initialize_forest(frequencies: list[int]) -> list[Node]:
    """
    Initializes a forest (list) of Node objects for each character with a non-zero frequency.
    """
    forest = []
    for i, freq in enumerate(frequencies):
        if freq >0:# create a node for all non zeros 
            symbol = chr(i+ord("A")) if i <26 else " "
            forest.append(Node(freq,symbol))#created a leaf node to add to the forest 
    return forest 
    


def build_huffman_tree(frequencies: list[int]) -> Node:
    """
    Builds the Huffman tree from the list of frequencies and returns the root Node.
    """
    forest = initialize_forest(frequencies)
    # Your code here
    while len(forest) > 1:#keep merging untill only the root is left 
        forest.sort()  # uses lt from Node
        #remove the two smallest nodes 
        left = forest.pop(0)
        right = forest.pop(0)
        #create a new node combing them and keep doing this until the root is the only one left 
        merged = Node(left.get_frequency() + right.get_frequency())
        merged.set_left(left)
        merged.set_right(right)
        #add the new node back to the forest 
        forest.append(merged)

    return forest


def build_encoding_table(huffman_tree_root: Node) -> list[str]:
    """
    Builds the encoding table from the Huffman tree.
    Returns a list of 27 strings, where index 0-25 correspond to 'A'-'Z'
    and index 26 corresponds to space.
    Each string is the binary encoding for that character.
    """
    encoding_table = [""] * 27
        #recursively traverse 
    def traverse(node: Node, code: str):
        if node is not None:
            if node.get_symbol() is not None:  # leaf
                symbol = node.get_symbol()
                index = 26 if symbol == " " else ord(symbol) - ord("A")
                encoding_table[index] = code #assign the code to this character 
            else:
                #traverse left with 0 and right with 1 
                traverse(node.get_left(), code + "0")
                traverse(node.get_right(), code + "1")

    traverse(huffman_tree_root, "")
    return encoding_table


def encode(input_string: str, encoding_table: list[str]) -> str:
    """
    Encodes the input string using the provided encoding table. Remember
    that the encoding table has 27 entries, one for each letter A-Z and
    one for space. Space is at the last index (26).
    """
    #reaplaces each character with its code 
    encoded =""
    for char in input_string:
        if char =="":
            encoded += encoding_table[26]
        else:
            index = ord(char)- ord("A")
            encoded += encoding_table[index]

    return encoded


def decode(encoded_string: str, huffman_root: Node) -> str:
    """
    Decodes the encoded string using the Huffman table as a key.
    """
    #start at the root of the tree and traverse acording to the bits in the encoded string
    decoded = ""
    current = huffman_root
    for bit in encoded_string:
        if bit ==0:
            current = current.get_left()
        else:
            current = current.get_right()
        if current.get_symbol() is not None:#found a character 
            decoded += current.get_symbol
            current = huffman_root#restart from root for the nect character

    return decoded 
   